In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch

import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.datasets as dataset

from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import os
import numpy as np

## Resizing images

#### This part has been implemented but if you have to resize the images again uncomment and store in a separate location

In [11]:
sz = (300,300) #specify the final size

In [12]:
# for n in fn2:
# #     set_trace()
# #     print(n)
#     im = Image.open(f'data/condition/{n}')
#     im.thumbnail(sz,Image.ANTIALIAS)
#     im.save(f'data/montgomery_china_resized/condition/{n}',format='png')

In [13]:
# for n in fn1:
# #     set_trace()
# #     print(n)
#     im = Image.open(f'data/normal/{n}')
#     im.thumbnail(sz,Image.ANTIALIAS)
#     im.save(f'data/montgomery_china_resized/normal/{n}',format='png')
    

---

## Data Loader

In [14]:
# Hyper Parameters
num_epochs = 10
batch_size = 16
learning_rate = 0.001

**This method of data loader is used if we have 2 folders train and validation and there are separate folders for each class within them**

In [15]:
# Data loading code
transform = transforms.Compose([
    transforms.RandomSizedCrop(290),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
# #     transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
# #                          std = [ 0.229, 0.224, 0.225 ]),
])

* transform normalize are required if we are using pre trained networks and we need to apply the same mean and std from the dataset it was trained on

In [20]:
#for training
traindir = os.path.join('data/montgomery_china_resized/', 'train')
train = dataset.ImageFolder(traindir, transform)
train_loader = torch.utils.data.DataLoader(
train, batch_size=batch_size, shuffle=True)

In [21]:
#for validation
valdir = os.path.join('data/montgomery_china_resized/', 'valid')
valid = dataset.ImageFolder(valdir, transform)
valid_loader = torch.utils.data.DataLoader(
valid, batch_size=batch_size, shuffle=False)

---

## Building your CNN network

In [22]:

# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(72*72*32, 2)
        
    def forward(self, x):
#         set_trace()
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [23]:
cnn = CNN() #instantiating your CNN class
cnn.cuda() #shifting it to gpu

criterion = nn.CrossEntropyLoss() #defining your loss
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate) #defining your optimizer

---
## Training of the network

In [27]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
#         set_trace()
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train)//batch_size, loss.data[0]))

Epoch [1/10], Iter [10/42] Loss: 0.6801
Epoch [1/10], Iter [20/42] Loss: 0.5891
Epoch [1/10], Iter [30/42] Loss: 0.7062
Epoch [1/10], Iter [40/42] Loss: 0.9009
Epoch [2/10], Iter [10/42] Loss: 0.5291
Epoch [2/10], Iter [20/42] Loss: 0.5644
Epoch [2/10], Iter [30/42] Loss: 0.6723
Epoch [2/10], Iter [40/42] Loss: 0.6372
Epoch [3/10], Iter [10/42] Loss: 0.7563
Epoch [3/10], Iter [20/42] Loss: 0.8041
Epoch [3/10], Iter [30/42] Loss: 0.6315
Epoch [3/10], Iter [40/42] Loss: 0.5619
Epoch [4/10], Iter [10/42] Loss: 0.6482
Epoch [4/10], Iter [20/42] Loss: 0.5227
Epoch [4/10], Iter [30/42] Loss: 0.5380
Epoch [4/10], Iter [40/42] Loss: 1.0973
Epoch [5/10], Iter [10/42] Loss: 0.5827
Epoch [5/10], Iter [20/42] Loss: 0.6955
Epoch [5/10], Iter [30/42] Loss: 0.7515
Epoch [5/10], Iter [40/42] Loss: 0.7205
Epoch [6/10], Iter [10/42] Loss: 0.5126
Epoch [6/10], Iter [20/42] Loss: 0.6561
Epoch [6/10], Iter [30/42] Loss: 0.6456
Epoch [6/10], Iter [40/42] Loss: 0.6481
Epoch [7/10], Iter [10/42] Loss: 0.6613


---
## Predicting on validation set

In [28]:
cnn.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in valid_loader:
    images = Variable(images).cuda()
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn.pkl')

Test Accuracy of the model on the 10000 test images: 67 %


---

## Custom dataloader

In [7]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

### Folder structure for using this data loader:
* All your images are in one folder that you'll pass as data_folder
* All the labels for the images are in a file called labels.csv
* directory structure: data_folder and labels.csv are inside root_dir
* index: for training/validation purpose...it should be a list of indices

In [44]:
class Dataset_from_csv(Dataset):
    

    def __init__(self, label_file, root_dir, data_folder,transform,index):
        """
        Args:
            label_file (string): Name of the label file which contains id,class.
            root_dir (string): Directory which contains the labels file and data folder.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            data_folder(string): name of the folder containing the images
            index (list): list of indices to pick from the labels file. It's useful when you have to pick train and validation
        """ 
        self.root_dir = root_dir
        self.index = index
        self.labels = pd.read_csv(f'{root_dir}{label_file}').iloc[self.index]
        self.classes, self.class_to_idx = self.find_classes()
        self.data_folder = data_folder
        self.imgs = self.make_dataset()
        self.transform = transform
            
    def find_classes(self):
        classes = list(self.labels.iloc[:,1].unique())
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def make_dataset(self): 
        path = self.labels.iloc[:,0].apply(lambda x: os.path.join(self.root_dir,self.data_folder,x))
        target = self.labels.iloc[:,1].apply(lambda x: self.class_to_idx[x])
        images = list(zip(path,target))

        return images
    
    def pil_loader(self,path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        path,target = self.imgs[idx]
        img = self.pil_loader(path)
        if self.transform is not None:
            img = self.transform(img)
        return img, target



In [45]:
train_idx = list(range(1000))
val_idxs = list(range(1001,2000))

In [46]:
train = Dataset_from_csv(data_folder='images_transform/',index=train_idx,label_file='nih_label_normal_condition.csv',transform=transform,root_dir='data/')
train_loader = torch.utils.data.DataLoader(
train, batch_size=batch_size, shuffle=True)

In [47]:
valid = Dataset_from_csv(data_folder='images_transform/',index=val_idxs,label_file='nih_label_normal_condition.csv',transform=transform,root_dir='data/')
valid_loader = torch.utils.data.DataLoader(
valid, batch_size=batch_size, shuffle=False)